In [1]:
import sys
sys.path.insert(1, '/home/afkirby/Analysis')
import Geometry as geo
from Transform import getVel3D
from Flow import flowLine3D, isHorseshoe
import numpy as np
from read_penguin import load_3D_data, cell_center
from matplotlib import pyplot as plt
from time import time

In [2]:
#Loading Data
xres3d = 288
yres3d = 480
zres3d = 144
data3d = load_3D_data("/scratch/afkirby/3DRsVariation/rs001/", xres3d, yres3d, zres3d, "h50_1p10E_e0_PPM4_ave", 10)

In [3]:
planetCoords = (1,np.pi,np.pi/2)
xp, yp, zp = geo.sphericalToCartesian(planetCoords, dim = 3)
planetVel = (0,1,0)

def isoVel3D(x, y):
    coords = geo.cartesianToSpherical(y, dim = 3)
    v = getVel3D(coords, planetCoords, planetVel, data3d, cart = True)
    return v

In [4]:
def velMag(v):
    return np.sqrt(v[0]**2 + v[1]**2 + v[2]**2)

In [6]:
t1 = time()
v = velMag(isoVel3D(0, (0,0,0)))
print(time() - t1)

0.00040340423583984375
